In [7]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
from pathlib import Path

In [27]:
#Selecionando as Tábuas e Variáveis
tabua_mortalidade = 'AMERICAN EXPERIENCE'
tabua_mortalidade_inv = 'RRB-44'
tabua_entrada_inv = 'ALVARO VINDAS'
tabua_morbidez = 'Atuários Ingleses'
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

In [9]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_INV'] = pd.to_datetime(bd_ativos['DT_NASC_INV'], yearfirst=True, format='%d/%m/%Y')

bd_ativos['DT_BASE'] = '31/12/2020'
bd_ativos['DT_BASE'] = pd.to_datetime(bd_ativos.DT_BASE, yearfirst=True, format='%d/%m/%Y')

In [13]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = round((bd_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_CONJUGE = round((bd_ativos.DT_BASE - statis_ativos.IDADE_CONJUGE) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades dos Cônjuges | Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5) | (statis_ativos.IDADE_CONJUGE > 127), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_EMT'] = round(statis_ativos.IDADE_ADM - (bd_ativos['NU_TEMPO_RGPS']/365.25)) #Calculando a Idade de Entrada no Mercado de Trabalho | Idade de Admissão - (TSANT/365.25)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos.IDADE_CONJUGE - statis_ativos.IDADE_SERVIDOR #Calculando a Diferença Etária do Cônjuge | Idade do Cônjuge - Idade do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos.DT_NASC_NOVO.mask(bd_ativos.DT_NASC_NOVO.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_NOVO = round((bd_ativos.DT_BASE - statis_ativos.IDADE_NOVO) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Mais Novo | Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos.IDADE_NOVO > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_INV'] = bd_ativos.DT_NASC_INV.mask(bd_ativos.DT_NASC_INV.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_INV = round((bd_ativos.DT_BASE - statis_ativos.IDADE_INV) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Inválido | Data-Base - Data de Nascimento do Filho Inválido
statis_ativos.loc[statis_ativos.IDADE_INV > 127, 'IDADE_INV'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['TP_PREFEITURA'] = statis_ativos.IDADE_SERVIDOR - statis_ativos.IDADE_ADM #Calculando o Tempo de Prefeitura | Idade do Servidor - Idade de Admissão
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIFERIMENTO'] = bd_ativos.DT_PROV_APOSENT - statis_ativos.IDADE_SERVIDOR #Calculando o Diferimento até a Aposentadoria | Idade Provável de Aposentadoria - Idade do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['BENEF_PROJ'] = round(bd_ativos.VL_BASE_CALCULO * (1 + cres_sal)**statis_ativos.DIFERIMENTO, 2)

#Idade de entrada no plano ou RPPS
#Perguntar a rúniu sobre a IDADE_EMT se considera o tempo do RPPS também

statis_ativos#.head(20)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_EMT,DIF_ETARIA,IDADE_NOVO,IDADE_INV,TP_PREFEITURA,DIFERIMENTO,BENEF_PROJ
0,40,20,NaN,20.0,NaN,16.0,NaN,20,20,1342.21
1,45,42,48.0,25.0,3.0,NaN,NaN,3,17,1302.73
2,48,27,NaN,25.0,NaN,18.0,NaN,21,14,1964.27
3,31,23,NaN,23.0,NaN,NaN,NaN,8,31,4599.23
4,42,35,NaN,25.0,NaN,NaN,NaN,7,23,1382.88
...,...,...,...,...,...,...,...,...,...,...
357,45,22,NaN,22.0,NaN,NaN,NaN,23,12,2716.30
358,26,24,NaN,24.0,NaN,NaN,NaN,2,31,4046.86
359,36,25,NaN,25.0,NaN,NaN,NaN,11,24,4223.40
360,44,30,NaN,25.0,NaN,NaN,NaN,14,16,2511.36


In [11]:
#Projeção do Crescimento Salarial (Benefício de Aposentadoria Projetado)
proj_cres_sal = []
for l in range(len(bd_ativos)):
    linha = []
    for c in range(statis_ativos.DIFERIMENTO[l]+1):
        benef_proj = round(bd_ativos.VL_BASE_CALCULO.loc[l] * ((1 + cres_sal)**c), 2)
        linha.append(benef_proj)
    proj_cres_sal.append(linha)

df_benef_proj = pd.DataFrame(data=proj_cres_sal, index=bd_ativos.ID_SERVIDOR_MATRICULA)

In [30]:
#Importando as Funções Biométricas
qx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx', usecols=[tabua_mortalidade])
px = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='px', usecols=[tabua_mortalidade])
lx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='lx', usecols=[tabua_mortalidade])
dx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='dx', usecols=[tabua_mortalidade])

In [48]:
qx['AMERICAN EXPERIENCE']

0      0.15470
1      0.06349
2      0.03550
3      0.02391
4      0.01770
        ...   
122    1.00000
123    1.00000
124    1.00000
125    1.00000
126    1.00000
Name: AMERICAN EXPERIENCE, Length: 127, dtype: float64

In [61]:
df_func_biometricas = pd.DataFrame()

df_func_biometricas['TESTE'] = statis_ativos.IDADE_EMT
#df_func_biometricas['TESTE2'] = df_func_biometricas.apply(lambda x: x['TESTE']+100, axis=1)
df_func_biometricas['qx'] = df_func_biometricas.TESTE.map(qx[tabua_mortalidade])
df_func_biometricas

,TESTE,qx
0,20.0,0.00780
1,25.0,0.00806
2,25.0,0.00806
3,23.0,0.00796
4,25.0,0.00806
...,...,...
357,22.0,0.00791
358,24.0,0.00801
359,25.0,0.00806
360,25.0,0.00806


In [53]:
df_func_biometricas.TESTE3.sum()

2.9002

In [12]:
statis_ativos[statis_ativos['IDADE_CONJUGE'].notnull()].head()

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_EMT,DIF_ETARIA,IDADE_NOVO,IDADE_INV,TP_PREFEITURA,DIFERIMENTO,BENEF_PROJ
1,45,42,48.0,25.0,3.0,NaN,NaN,3,17,1302.7349
6,40,22,53.0,22.0,13.0,NaN,NaN,18,21,1355.6311
11,41,26,44.0,25.0,3.0,NaN,NaN,15,24,1396.7081
17,53,30,30.0,25.0,-23.0,3.0,NaN,23,5,4574.6369
23,40,28,54.0,24.0,14.0,16.0,NaN,12,25,1410.6752
